In [ ]:
import os
import gc 
from glob import glob
from pathlib import Path
from datetime import datetime
import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import lightgbm
import torch
import torch.nn as nn

from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier, Pool
import xgboost
from collections import Counter

import shap
import pickle
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
ROOT            = Path("/kaggle/input/home-credit-risk-model-train-test")
SCHEMA_PATH     = Path("/kaggle/input/schema-home-credit-risk-data")
MODEL           = Path("/kaggle/input/homecredit-risk-model-stability/other/catboost_lgb_champ/3")

In [ ]:
def set_data_types(df, schema):
    
    for col, dtype in schema.items():
        if dtype == 'category':
            dtype = 'object'
        df[col] = df[col].astype(dtype, errors = 'ignore')
    return df

In [ ]:
X = pl.read_csv(ROOT/'model_abt_train.csv').to_pandas()
X_test = pl.read_csv(ROOT/'model_abt_test.csv').to_pandas()

schema = pd.read_csv(SCHEMA_PATH / "data_schema.csv", names = ['Columns', 'dtype'])
schema = schema[schema['Columns'].isin(X.columns)]
schema = schema.set_index('Columns')['dtype'].to_dict()

X = set_data_types(X, schema)
X_test = set_data_types(X_test, schema)
del schema

In [ ]:
id_val = X[['case_id', 'WEEK_NUM']]
id_test_val = X_test[['case_id', 'WEEK_NUM']]

y = X['target']
y_test = X_test['target']

cols = X.drop(['case_id', 'WEEK_NUM', 'target', 'decision_month', 'decision_weekday'], axis = 1).columns

X = X.drop(['case_id', 'WEEK_NUM', 'target', 'decision_month', 'decision_weekday'], axis = 1)
X_test = X_test.drop(['case_id', 'WEEK_NUM', 'target', 'decision_month', 'decision_weekday'], axis = 1)

In [ ]:
with open(ROOT/'cat_cols.pkl', 'rb') as f:
    cat_cols = pickle.load(f)
num_cols = [x for x, dtypes in X.dtypes.items() if dtypes in ['float', 'int']]
X[cat_cols] = X[cat_cols].astype(str)
X_test[cat_cols] = X_test[cat_cols].astype(str)

In [ ]:
class Model_Utils:
    @staticmethod
    def model_evals(y_true, y_proba, cutoff = 0.5):
        from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, average_precision_score, roc_auc_score
        """
        Returns model evaluation metrics for a binary classification model

        Parameters:
        -----------
            y_true: int (0,1) 
                Actual binary labels

            y_proba: float (between 0 and 1)
                Probability scores output of model 

        Returns:
        --------
            result: dict
                Dictionary of metrics and their results based on the input
                    - event rate (% predicted 1's)
                    - accuracy
                    - roc_auc
                    - pr_auc
                    - recall
                    - precision
                    - f1
                    - lift
        """

        y_pred = (y_proba > cutoff).astype(int)

        event_rate = y_pred.mean()

        accuracy = accuracy_score(y_true, y_pred)

        roc_auc = roc_auc_score(y_true, y_proba)

        pr_auc = average_precision_score(y_true, y_proba)

        recall = recall_score(y_true, y_pred)

        precision = precision_score(y_true, y_pred)

        f1 = f1_score(y_true, y_pred)

        lift = recall / event_rate

        return {'event_rate': event_rate,
                'acc': accuracy, 
                'precision': precision, 
                'recall': recall, 
                'f1': f1, 
                'roc_auc': roc_auc,
                'pr_auc': pr_auc, 
                'lift': lift}
    
    @staticmethod
    def cutoff_perc(y_actual, y_proba, percent):
        n_cutoff = int(len(y_actual) * percent)

        scores = pd.DataFrame({'actual': y_actual, 'proba': y_proba})
        scores = scores.sort_values(by = 'proba', ascending= False)
        scores['rank'] = scores['proba'].rank(ascending = False)
        scores['ranked_pred'] = np.where(scores['rank']<= n_cutoff, 1, 0)

        print(len(scores[scores['ranked_pred'] == 1]))
        print(scores[scores['ranked_pred'] == 1]['proba'].min())
        return Model_Utils.model_evals(y_actual, y_proba, cutoff = scores[scores['ranked_pred'] == 1]['proba'].min())
    
    @staticmethod
    def DumbClassifier(y_true):
        import numpy as np
        y_pred = np.zeros(len(y_true))
        return y_pred
    
    @staticmethod
    def RandomChanceClassifier(y_true):
        import numpy as np
        y_pred = Model_Utils.DumbClassifier(y_true)
        event = y_true.sum()
        ind = np.random.randint(0, len(y_true), size=event)
        y_pred[ind] = 1
        return y_pred
    
    @staticmethod
    def save_model_results(results, schema, filepath):
        if not os.path.exists(filepath):
            score_schema = {'model_name': 'str', 
                        'model': 'str',
                        'params': 'str',
                        'acc': 'float',
                        'precision': 'float',
                        'recall': 'float',
                        'f1_score': 'float',
                        'roc_auc': 'float',
                        'pr_auc': 'float',
                        'lift': 'float'}
            scores = pd.DataFrame(columns = score_schema.keys()).astype(score_schema)
        else:
            scores = pd.read_csv(filepath)
        row = []
        
        for key in schema.keys():
            row.append(results[key])
        
        scores.loc[len(scores)] = row
        scores.to_csv(filepath, index = False)
        
    @staticmethod
    def LightGBMClassifier_CV(X, y, cat_cols, cv = 5, group = None, params = None):
        import lightgbm
        from sklearn.model_selection import StratifiedGroupKFold
        
        cv = StratifiedGroupKFold(n_splits=cv, shuffle = True, random_state = 42)
        
        if params == None:
            params = {'random_state': 42
                     ,'objective': 'binary'
                     ,'verbose': -1
                     ,'n_jobs': -1}
        
        scores = {'acc': [],
                        'precision': [],
                        'recall': [],
                        'f1': [],
                        'roc_auc': [],
                        'pr_auc': [],
                        'lift': []}

        split = 1
        
        for train_ind, valid_ind in cv.split(X, y, groups=group):
            X_train, y_train = X.iloc[train_ind], y.iloc[train_ind]
            X_valid, y_valid = X.iloc[valid_ind], y.iloc[valid_ind]

            X_train[cat_cols] = X_train[cat_cols].astype("category")
            X_valid[cat_cols] = X_valid[cat_cols].astype("category")
            
            lgb = lightgbm.LGBMClassifier(**params)
            lgb.fit(X_train, y_train)

            y_proba = lgb.predict_proba(X_valid)[:, 1]
            results = Model_Utils.model_evals(y_valid, y_proba)
            print(f'LightGBM, Val CV{split}: {results}"')
            
            split += 1
            
            for key in results.keys():
                if key in scores.keys():
                    scores[key].append(results[key])
        
        for key in results.keys():
            if key in scores.keys():
                scores[key] = np.array(scores[key]).mean()
            
        print(f'LightGBM, Average CV: {scores}"')
        return lgb, scores

    
    @staticmethod
    def LightGBMClassifier_pred(X, y, X_test, y_test, cat_cols, params = None):
        import lightgbm
        
        if params == None:
            params = {'random_state': 42
                     ,'objective': 'binary'
                     ,'verbose': -1
                     ,'n_jobs': -1}
        
        scores = {'acc': [],
                        'precision': [],
                        'recall': [],
                        'f1': [],
                        'roc_auc': [],
                        'pr_auc': [],
                        'lift': []}

        
        X[cat_cols] = X[cat_cols].astype("category")
        X_test[cat_cols] = X_test[cat_cols].astype("category")
            
        lgb = lightgbm.LGBMClassifier(**params)
        lgb.fit(X, y)

        y_proba = lgb.predict_proba(X_test)[:, 1]
        results = Model_Utils.model_evals(y_test, y_proba)
        print(f'LightGBM, Test: {results}"')
        
        return lgb, y_proba, results
    
    
    @staticmethod
    def CatBoostClassifier_CV(X, y, cat_cols, cv = 5, group = None, params = None):
        import catboost
        from sklearn.model_selection import StratifiedGroupKFold
        
        cv = StratifiedGroupKFold(n_splits=cv, shuffle = True, random_state = 42)
        
        if params == None:
             params = {'boosting_type' : "Plain",
                      'eval_metric': 'PRAUC',
                      'random_seed': 42,
                      'learning_rate': 0.05,
                      'use_best_model': True,
                      'iterations': 1000}
        
        scores = {'acc': [],
                        'precision': [],
                        'recall': [],
                        'f1': [],
                        'roc_auc': [],
                        'pr_auc': [],
                        'lift': []}

        split = 1
        
        for train_ind, valid_ind in cv.split(X, y, groups=group):
            X_train, y_train = X.iloc[train_ind], y.iloc[train_ind]
            X_valid, y_valid = X.iloc[valid_ind], y.iloc[valid_ind]
            
            
            clf = CatBoostClassifier(**params)
        
            train_pool = Pool(X_train, y_train, cat_features=cat_cols)
            val_pool = Pool(X_valid, y_valid, cat_features=cat_cols)
            
            clf.fit(train_pool, eval_set=val_pool, verbose=False)
            
            y_proba = clf.predict_proba(X_valid)[:, 1]
            results = Model_Utils.model_evals(y_valid, y_proba)
            print(f'CatBoost, Val CV{split}: {results}"')
            
            split += 1
            
            for key in results.keys():
                if key in scores.keys():
                    scores[key].append(results[key])
        
        for key in results.keys():
            if key in scores.keys():
                scores[key] = np.array(scores[key]).mean()
            
        print(f'CatBoost, Average CV: {scores}"')
        return clf, scores
    
    
    @staticmethod
    def CatBoostClassifier_pred(X, y, X_test, y_test, cat_cols, params = None):
        import catboost
    
        if params == None:
             params = {'boosting_type' : "Plain",
                      'eval_metric': 'PRAUC',
                      'random_seed': 42,
                      'learning_rate': 0.05,
                      'use_best_model': True,
                      'iterations': 1000}
                
        clf = CatBoostClassifier(**params)
        
        train_pool = Pool(X, y, cat_features=cat_cols)
        test_pool = Pool(X_test, y_test, cat_features=cat_cols)
            
        clf.fit(train_pool, eval_set=test_pool, verbose=False)
            
        y_proba = clf.predict_proba(X_test)[:, 1]
        results = Model_Utils.model_evals(y_test, y_proba)
        print(f'CatBoost, Test: {results}"')
            
        return clf, y_proba, results

    
    @staticmethod
    def train_test_split(X, y, test_size = 0.2):
        from sklearn.model_selection import train_test_split
        import polars as pl
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
        pl.from_pandas(pd.concat([y_train, X_train], axis = 1)).write_csv('model_abt_train.csv')
        del X_train
        del y_train
        gc.collect()
        
        pl.from_pandas(pd.concat([y_test, X_test], axis = 1)).write_csv('model_abt_test.csv')
        del X_test
        del y_test
        gc.collect()

In [ ]:
params_lgb = {'colsample_bytree': 0.8091363920278533, 
              'gamma': 1.6740930883959255, 
              'learning_rate': 0.0816628382148218, 
              'max_depth': 7, 'min_child_weight': 1.7065771300111194, 
              'n_estimators': 250, 'random_state': 42, 
              'reg_alpha': 7.114616964626139, 'reg_lambda': 8.853463059493436, 
              'subsample': 0.5494975675591438, 'verbose': -1}


params_catboost = {'bagging_temperature': 3.0469773840224983,
                   'boosting_type': 'Ordered', 
                   'border_count': 96, 
                   'colsample_bylevel': 0.5008191566781925, 
                   'depth': 5, 'iterations': 250, 
                   'l2_leaf_reg': 8.0, 
                   'learning_rate': 0.19362752313765308, 
                   'max_ctr_complexity': 5.0, 
                   'one_hot_max_size': 50, 
                   'random_state': 42, 
                   'verbose': -1}

In [ ]:
X.shape

In [ ]:
X_test.shape

In [ ]:
cat_cols_all = cat_cols

In [ ]:
lgb_imp = pd.read_csv(MODEL/"LightGBM_Feature_Imp.csv")
cat_imp = pd.read_csv(MODEL/"CatBoost_Feature_Imp.csv")
lgb_imp['Cum_Imp'] = lgb_imp['perc_imp'].cumsum()
cat_imp['Cum_Imp'] = cat_imp['perc_imp'].cumsum()

#### Keepp 99% of Cumulative Feature Importance

In [ ]:
rfe_cols = list(lgb_imp[lgb_imp['Cum_Imp']>0.99]['0'])
cols = [x for x in X.columns if x not in rfe_cols]
cat_cols = [x for x in cat_cols_all if x in cols]
print(len(cols))
lgb, lgb_pred, lgb_scores = Model_Utils.LightGBMClassifier_pred(X[cols], y, X_test[cols], y_test, cat_cols, params_lgb)
with open('lgb_train_rfe_num_99.joblib', 'wb') as f:
    joblib.dump(lgb, f)

In [ ]:
rfe_cols = list(cat_imp[cat_imp['Cum_Imp']>0.99]['0'])
cols = [x for x in X.columns if x not in rfe_cols]
cat_cols = [x for x in cat_cols_all if x in cols]
print(len(cols))
clf, y_pred, cat_scores = Model_Utils.CatBoostClassifier_pred(X[cols], y, X_test[cols], y_test, cat_cols, params_catboost)
with open('catboost_train_rfe_num_99.joblib', 'wb') as f:
    joblib.dump(clf, f)

#### Keep 95% of Cumulative Feature Importance

In [ ]:
rfe_cols = list(lgb_imp[lgb_imp['Cum_Imp']>0.95]['0'])
cols = [x for x in X.columns if x not in rfe_cols]
cat_cols = [x for x in cat_cols_all if x in cols]
print(len(cols))
lgb, lgb_pred, lgb_scores = Model_Utils.LightGBMClassifier_pred(X[cols], y, X_test[cols], y_test, cat_cols, params_lgb)
with open('lgb_train_rfe_num_95.joblib', 'wb') as f:
    joblib.dump(lgb, f)

In [ ]:
rfe_cols = list(cat_imp[cat_imp['Cum_Imp']>0.95]['0'])
cols = [x for x in X.columns if x not in rfe_cols]
cat_cols = [x for x in cat_cols_all if x in cols]
print(len(cols))
clf, y_pred, cat_scores = Model_Utils.CatBoostClassifier_pred(X[cols], y, X_test[cols], y_test, cat_cols, params_catboost)
with open('catboost_train_rfe_num_95.joblib', 'wb') as f:
    joblib.dump(clf, f)

#### Keep 90% if Cumulative Feature Importance

In [ ]:
rfe_cols = list(lgb_imp[lgb_imp['Cum_Imp']>0.9]['0'])
cols = [x for x in X.columns if x not in rfe_cols]
cat_cols = [x for x in cat_cols_all if x in cols]
print(len(cols))
lgb, lgb_pred, lgb_scores = Model_Utils.LightGBMClassifier_pred(X[cols], y, X_test[cols], y_test, cat_cols, params_lgb)
with open('lgb_train_rfe_num_90.joblib', 'wb') as f:
    joblib.dump(lgb, f)

In [ ]:
rfe_cols = list(cat_imp[cat_imp['Cum_Imp']>0.90]['0'])
cols = [x for x in X.columns if x not in rfe_cols]
cat_cols = [x for x in cat_cols_all if x in cols]
print(len(cols))
clf, y_pred, cat_scores = Model_Utils.CatBoostClassifier_pred(X[cols], y, X_test[cols], y_test, cat_cols, params_catboost)
with open('catboost_train_rfe_num_90.joblib', 'wb') as f:
    joblib.dump(clf, f)

#### Keep 80% of Cumulative Feature Importance

In [ ]:
rfe_cols = list(lgb_imp[lgb_imp['Cum_Imp']>0.8]['0'])
cols = [x for x in X.columns if x not in rfe_cols]
cat_cols = [x for x in cat_cols_all if x in cols]
print(len(cols))
lgb, lgb_pred, lgb_scores = Model_Utils.LightGBMClassifier_pred(X[cols], y, X_test[cols], y_test, cat_cols, params_lgb)
with open('lgb_train_rfe_num_80.joblib', 'wb') as f:
    joblib.dump(lgb, f)

In [ ]:
rfe_cols = list(cat_imp[cat_imp['Cum_Imp']>0.8]['0'])
cols = [x for x in X.columns if x not in rfe_cols]
cat_cols = [x for x in cat_cols_all if x in cols]
print(len(cols))
clf, y_pred, cat_scores = Model_Utils.CatBoostClassifier_pred(X[cols], y, X_test[cols], y_test, cat_cols, params_catboost)
with open('catboost_train_rfe_num_80.joblib', 'wb') as f:
    joblib.dump(clf, f)

#### Keep 70% of Cumulative Feature Importance

In [ ]:
rfe_cols = list(lgb_imp[lgb_imp['Cum_Imp']>0.7]['0'])
cols = [x for x in X.columns if x not in rfe_cols]
cat_cols = [x for x in cat_cols_all if x in cols]
print(len(cols))
lgb, lgb_pred, lgb_scores = Model_Utils.LightGBMClassifier_pred(X[cols], y, X_test[cols], y_test, cat_cols, params_lgb)
with open('lgb_train_rfe_num_70.joblib', 'wb') as f:
    joblib.dump(lgb, f)

In [ ]:
rfe_cols = list(cat_imp[cat_imp['Cum_Imp']>0.7]['0'])
cols = [x for x in X.columns if x not in rfe_cols]
cat_cols = [x for x in cat_cols_all if x in cols]
print(len(cols))
clf, y_pred, cat_scores = Model_Utils.CatBoostClassifier_pred(X[cols], y, X_test[cols], y_test, cat_cols, params_catboost)
with open('catboost_train_rfe_num_70.joblib', 'wb') as f:
    joblib.dump(clf, f)

#### Keep 60% of Cumulative Feature Importance

In [ ]:
rfe_cols = list(lgb_imp[lgb_imp['Cum_Imp']>0.6]['0'])
cols = [x for x in X.columns if x not in rfe_cols]
cat_cols = [x for x in cat_cols_all if x in cols]
print(len(cols))
lgb, lgb_pred, lgb_scores = Model_Utils.LightGBMClassifier_pred(X[cols], y, X_test[cols], y_test, cat_cols, params_lgb)
with open('lgb_train_rfe_num_60.joblib', 'wb') as f:
    joblib.dump(lgb, f)

In [ ]:
rfe_cols = list(cat_imp[cat_imp['Cum_Imp']>0.6]['0'])
cols = [x for x in X.columns if x not in rfe_cols]
cat_cols = [x for x in cat_cols_all if x in cols]
print(len(cols))
clf, y_pred, cat_scores = Model_Utils.CatBoostClassifier_pred(X[cols], y, X_test[cols], y_test, cat_cols, params_catboost)
with open('catboost_train_rfe_num_60.joblib', 'wb') as f:
    joblib.dump(clf, f)

#### Keep 50% of Cumulative Feature Importance

In [ ]:
rfe_cols = list(lgb_imp[lgb_imp['Cum_Imp']>0.5]['0'])
cols = [x for x in X.columns if x not in rfe_cols]
cat_cols = [x for x in cat_cols_all if x in cols]
print(len(cols))
lgb, lgb_pred, lgb_scores = Model_Utils.LightGBMClassifier_pred(X[cols], y, X_test[cols], y_test, cat_cols, params_lgb)
with open('lgb_train_rfe_num_50.joblib', 'wb') as f:
    joblib.dump(lgb, f)

In [ ]:
rfe_cols = list(cat_imp[cat_imp['Cum_Imp']>0.5]['0'])
cols = [x for x in X.columns if x not in rfe_cols]
cat_cols = [x for x in cat_cols_all if x in cols]
print(len(cols))
clf, y_pred, cat_scores = Model_Utils.CatBoostClassifier_pred(X[cols], y, X_test[cols], y_test, cat_cols, params_catboost)
with open('catboost_train_rfe_num_50.joblib', 'wb') as f:
    joblib.dump(clf, f)